# The `parameter` statement

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * A `parameter` statement defines a parameter that can be passed from command line
  * SoS determines type of parameters from default values or types
  * `--no-var` is used to specify `False` for boolean parameters

The `parameter` statement specifies variables that can be specified from command line (or magics such as `%run` from SoS Notebook).

The basic format of the statement is

```
parameter: name = value
```
where
* `name` is the name of the parameter
* `value` is either a default value, or a type (e.g. `str`, `int`). Parameters specified with types are required

Parameters are specified with double dash syntax from command line.

In [1]:
%run --var 10

parameter: var = 5
print(f'var is {var}')

var is 10


## Optional and required parameters

Parameters with default values are optional. The default value will be used if the parameter is left unspecified

In [2]:
parameter: var = 5
print(f'var is {var}')

var is 5

When a type is specified, a parameter will be required and an error will be raised if a required parameter is not specified

In [3]:
parameter: var = int
print(f'var is {var}')

[0]: Argument var of type int is required


In [ ]:
%run --var 10

parameter: var = int
print(f'var is {var}')

If you have a parameter with underscore `_`, it can be specified with either underscore (`_`) or dash (`-`).

In [ ]:
%run --input_files a.txt

parameter: input_files = list
print(input_files)

In [ ]:
%run --input-files a.txt b.txt

parameter: input_files = list
print(input_files)

## Type of parameters

The types of parameters are determined by the default value or type specification, and determines how they should be passed from command line.

### Simple Python types

SoS automatically determines the type of default values and convert your input data to the type. For example, the type of `cutoff` is determined to be an integer so it accepts an integer value from command line:

In [ ]:
%run --cutoff 5
parameter: cutoff = 0

print(cutoff)

An error will be raised if you pass a string,

In [ ]:
%run -v0 --cutoff zero
parameter: cutoff = 0

print(cutoff)

even a float value

In [ ]:
%run -v0 --cutoff 5.1
parameter: cutoff = 0

print(cutoff)

If you intended to accept an float value, use a default value in float:

In [ ]:
%run --cutoff 5.1
parameter: cutoff = 0.

print(cutoff)

### Yes or no (Boolean types)

Boolean data types can be specified using `--opt` and `--no-opt`:

In [ ]:
%run -v0 --test-mode

parameter: test_mode = bool
print(f'test_mode is {test_mode}')

In [ ]:
%run -v0 --no-test-mode

parameter: test_mode = bool
print(f'test_mode is {test_mode}')

This works the same for boolean variables with default value:

In [ ]:
%run -v0 

parameter: test_mode = False
print(f'test_mode is {test_mode}')

In [ ]:
%run -v0 --test-mode

parameter: test_mode = False
print(f'test_mode is {test_mode}')

### List of strings

A list would be created if the parameter has a default value of type list. For example, a list `['A']` is returned because the default value has a list type.

In [ ]:
%run --sample-names A
parameter: sample_names = []

print(sample_names)

SoS even understands the type of the values of the list and tries to follow it:

In [ ]:
%run --values 4 5
parameter: values = [1, 2, 3]

print(values)

However, it is not yet possible to specify the type of values when you specify a required parameter of type list so all the values will be passed as strings:

In [ ]:
%run --values 4 5
parameter: values = list

print(values)

### SoS types

SoS types such as `path`, `paths`, `file_targets`. Generally speaking you can pass filenames as `str` or list of `str`, but passing SoS types such as `path` allow you to create variable in these types without type coercion.

For example, the `path` type is derived from [`pathlib.Path`](https://docs.python.org/3.6/library/pathlib.html) with automatic expansion of `~` and other features. If you pass a parameter with type `path`, SoS will convert passed string into a `path` object so that you can use it directly.

In [ ]:
%run --infile ~/project/a.txt
parameter: infile = path

print(infile.name)
print(infile.parent)
print(infile.exists())

Similarly, if you pass a `paths` (a sequence of `path`), you parameter will be of type `paths`:

In [ ]:
%run --infiles a.txt b.txt
parameter: infiles = paths('a.txt')

print(infiles)

## Scope of parameters

Parameters are usually defined in the `global` section of workflows, but it can also be defined in steps, in which case the scope of the parameter is limited to the step in which it is defined.

The following example has `var_A` and `var_B` defined in steps `A` and `B`. Although `var_B` is specified with a type, it is not required when workflow `A` is executed.

In [ ]:
%run A --var_A 10

[A]
parameter: var_A = int
print(f'var A is {var_A}')

[B]
parameter: var_B = int
print(f'var B is {var_B}')

## Parameters and subworkflows

Parameters are usually specified from command line but it can be specified from a subworkflow as it the (nested) workflow is executed with specified parameter.

In [ ]:
%run

[sub]
parameter: val = 10
print(f'Calling workflow sub with parameter {val}')

[default]
sos_run('sub', val=5)
sos_run('sub', val=15)

The `%run` magic in this example executes the default workflow `default`, and executes the `sub` subworkflow with different parameters. You can also specify the workflow to execute directly.

In [ ]:
%run sub --val 20

[sub]
parameter: val = 10
print(f'Calling workflow sub with parameter {val}')

[default]
sos_run('sub', val=5)
sos_run('sub', val=15)

## Further reading <a id='further_reading'></a>

* [SoS Data Types](doc/user_guide/sos_datatypes.html)